In [4]:
import pandas as pd

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, udf
from pyspark.sql.types import ArrayType, IntegerType,  StringType

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StringIndexer

spark = SparkSession.builder.getOrCreate()

In [5]:
df = spark.read.csv("./jordan_subset", header=True)

In [6]:
df.show(5)

+------------+-------+-----------+---+---------+----------+------------+----------------------+-----------------+---------------+-------+--------------+------+-----+--------+---------------+-----+------------+--------------------+-------------+------+----------------+----------+
|subreddit_id|    _c0|created_utc|ups|  link_id|      name|score_hidden|author_flair_css_class|author_flair_text|      subreddit|     id|removal_reason|gilded|downs|archived|         author|score|retrieved_on|                body|distinguished|edited|controversiality| parent_id|
+------------+-------+-----------+---+---------+----------+------------+----------------------+-----------------+---------------+-------+--------------+------+-----+--------+---------------+-----+------------+--------------------+-------------+------+----------------+----------+
|    t5_2te6p|3345542| 1430609172|  4|t3_34mhlr|t1_cqwgwou|           0|                    NA|               NA|WaltDisneyWorld|cqwgwou|            NA|     0| 

In [7]:
df.count()

8799960

In [8]:
df.columns

['subreddit_id',
 '_c0',
 'created_utc',
 'ups',
 'link_id',
 'name',
 'score_hidden',
 'author_flair_css_class',
 'author_flair_text',
 'subreddit',
 'id',
 'removal_reason',
 'gilded',
 'downs',
 'archived',
 'author',
 'score',
 'retrieved_on',
 'body',
 'distinguished',
 'edited',
 'controversiality',
 'parent_id']

In [9]:
# tokenizer
# then stop words remover
# then countvectorizer
# word2vec??

In [10]:
tk = Tokenizer(inputCol="body", outputCol="words")
# tokenized = tk.transform(df)
# tokenized.select('body','words').show(5)

In [11]:
sw = StopWordsRemover(inputCol="words", outputCol="filtered")
# removedData = sw.transform(tokenized)
# removedData.select('body','words','filtered').show(5)

In [12]:
cv = CountVectorizer(inputCol="filtered", outputCol="counted", vocabSize=3, minDF=2.0)
# counted = cv.fit(removedData)
# counted.select('body','words','filtered','counted').show(5)

In [13]:
w2v =  Word2Vec(vectorSize=3, minCount=0, inputCol="filtered", outputCol="word2vec")
# vectored = w2v.fit(removedData)
# vectored.select('body','words','filtered',"word2vec").show(5)

In [14]:
si = StringIndexer(inputCol="subreddit_id", outputCol="sr_id_num")

In [15]:
ohe = OneHotEncoder(inputCol="sr_id_num", outputCol="subr_ohe")  

In [ ]:
# feats =  ['ups','gilded','score_hidden','downs','score','controversiality','counted','word2vec']
feats =  ['ups','gilded','score_hidden','downs','score','controversiality']
vecAs = VectorAssembler(inputCols=feats, outputCol="features")

In [16]:
# add this all to a pipeline before training the model

In [18]:
# pipeline = Pipeline(stages=[tk,sw,cv,w2v,ohe,vecAs])
pipeline = Pipeline(stages=[tk,sw,si,ohe,vecAs])
df_fitted = pipeline.fit(df)

In [ ]:
# Should we do an unsupervised analysis and pass in subreddits? Is there a supervised method we can use?